**Major Project: COMP8240**
For this Project, we will use the ariline_sentiment Dataset to train the fastText classifier replacing the original data of stack exchange questions. The project follows the workflow of application of data science project and we will use a facebook's fastText classifier for the classification of collected data from twitter and reddit.

Importing the airline_sentiment Dataset

In [1]:
import pandas as pd
df = pd.read_csv('airline_sentiment.csv')
df_check = pd.read_csv('airline_sentiment.csv')

Keeping the data of concern i.e. Tweets, sentiments and Twitter users

In [2]:
del df['airline_sentiment_confidence']
del df['airline_sentiment_gold']
del df['negativereason']
del df['negativereason_confidence']
del df['negativereason_gold']
del df['retweet_count']
del df['tweet_coord']
del df['tweet_created']
del df['tweet_id']
del df['tweet_location']
del df['user_timezone']
del df['airline']

In [3]:
import re
import numpy as np

Here comes the part of cleaning up the tweets before training the model on the airline_dataset. Because we want to replicate the original work of experiments, the data has to be of similar quality as of original data only the data annotations are changed to positive, negative an neutral.

In [4]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

The above funtion evaluates pyfunc over successive rows of the input arrays using the broadcasting rule of numpy.

In [5]:
df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")

In [6]:
df['text'] = np.vectorize(remove_pattern)(df['text'], "#")

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
max_tweets = 14639
corpus = []

To clean up the tweets we converted the tweet's text to lowercase, removed @ and Hashtags, removed website urls, removed special characters, removed white spaces, removed emojis/smileys.

In [9]:
for i in range(0, max_tweets):
    tweet = re.sub(r'(.*?)http.*?\s?(.*?)', '',
            df['text'][i], flags = re.MULTILINE)
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    tweet = tweet.lower()
    tweet = re.sub('rt', '', tweet)
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet
        if not word in set(stopwords.words('english'))]
    if tweet == []: continue
    else:
        tweet = ' '.join(tweet)
        corpus.append(tweet)

In [10]:
df['text']=df['text'].apply(lambda row:re.sub(r'http\S+', '',row))
df['text']=df['text'].apply(lambda row : row.lower())
df['text']=df['text'].apply(lambda row : re.sub('\d+','',row))
df['text']=df['text'].apply(lambda row:re.sub(r"[^A-Za-z0-9']+", ' ',row))
df['text']=df['text'].apply(lambda row:re.sub(r'\s+', ' ',row))

For the comparison between a random original tweet and cleaned tweet see below :

In [11]:
df.iloc[73].text

' your airline is awesome but your lax loft needs to step up its game for dirty tables and floors '

In [12]:
df_check.iloc[73].text

'@VirginAmerica your airline is awesome but your lax loft needs to step up its game. $40 for dirty tables and floors? http://t.co/hy0VrfhjHt'

Formatting the data to be read by fastText classifier. Labels must start by the prefix _label_ which is how fasttext recognizes what a label or what a word is. Csv is then futher converted into to a txt file which is then fed into the fastText training model.

In [13]:
df['airline_sentiment'] = '__label__' + df['airline_sentiment'].astype(str)
df['lableled']=df['airline_sentiment'].astype(str) +' '+ df['text']

In [14]:
del df['airline_sentiment']
del df['text']
del df['name']

In [15]:
df.head()

,lableled
0,__label__neutral what said
1,__label__positive plus you've added commercia...
2,__label__neutral i didn't today must mean i n...
3,__label__negative it's really aggressive to b...
4,__label__negative and it's a really big bad t...


In [16]:
df.to_csv('airline_labelled.csv', index = False, header=False)

In [17]:
import csv
csv_file = input('Input csv file name ')
txt_file = input('Input text file name')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Input csv file name airline_labelled.csv
Input text file nameairline_training.txt


Construction of new dataset using Tweepy a python library for accessing Twitter API

In [18]:
import tweepy
import json 

In [19]:
consumer_key = '8nJfxRD8HMRNPAjy1PqWX89cI'
consumer_key_secret = 'EQJEvLUisoH5DIAFBXZFqI4Du4QMMWIT7BGfsMx9D9s41fh2dI'
access_token = '1251336102285242369-evdUh0gDQzpkw6yXJTSQBBEajfzlIS'
access_token_secret = '8AJkJnuAMi97jNASBan7GsQi7flZM6c5HFvIcIZbx8fi2'

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)

auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

Since my goal was to provide frame an opinion on 2020 US presidential elections. I have pulled tweets from the twitter handle ‘US Politics Polls’

In [20]:
max_tweets = 200
screen_name = 'USPoliticsPoll'

In [21]:
tweets = api.user_timeline(screen_name = screen_name,
                            tweet_mode = 'extended',
                            count = max_tweets)

The function below writes tweets to a file:

In [22]:
list_of_dicts = []
for json_tweet in tweets:
    list_of_dicts.append(json_tweet._json)

In [23]:
with open('tweet_json_data.txt', mode = 'w') as file:
    file.write(json.dumps(list_of_dicts, indent = 4))

Reading Tweets from the file generated above.

In [24]:
list = []
with open('tweet_json_data.txt', encoding='utf-8', mode = 'r') as file:
    list_of_dicts = json.load(file)
    for dict in list_of_dicts:
    # Note we read selected information from the tweets.
        list.append({'tweet_id': str(dict['id']),
                'full_text': str(dict['full_text']),
                'favorite_count': int(dict['favorite_count']),
                'retweet_count': int(dict['retweet_count']),
                'created_at': dict['created_at'],
                })

Generating a dataframe using Tweets Information

In [25]:
import pandas as pd
df_new = pd.DataFrame(list, columns = ['tweet_id',
                                    'full_text',
                                    'favorite_count',
                                    'retweet_count',
                                    'created_at'])

Keeping the data of concern i.e. Tweets

In [26]:
del df_new['created_at']
del df_new['favorite_count']
del df_new['tweet_id']
del df_new['retweet_count']
df_new.rename(columns={'full_text': 'text'}, inplace=True)

Now cleaning up of the tweets and make them machine understandable.

In [27]:
def remove_patterns(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

In [28]:
df_new['text'] = np.vectorize(remove_patterns)(df_new['text'], "@[\w]*")

In [29]:
df_new['text'] = np.vectorize(remove_patterns)(df_new['text'], "#")

The process of cleaning is replicated as is done with the training dataset.

In [30]:
max_tweets = 200
corpus = []

In [31]:
for i in range(0, max_tweets):
    tweet = re.sub(r'(.*?)http.*?\s?(.*?)', '',
            df_new['text'][i], flags = re.MULTILINE)
    tweet = re.sub('[^a-zA-Z0-9]', ' ', tweet)
    tweet = tweet.lower()
    tweet = re.sub('rt', '', tweet)
    tweet = tweet.split()
    ps = PorterStemmer()
    tweet = [ps.stem(word) for word in tweet
        if not word in set(stopwords.words('english'))]
    if tweet == []: continue
    else:
        tweet = ' '.join(tweet)
        corpus.append(tweet)

In [32]:
df_new['text']=df_new['text'].apply(lambda row:re.sub(r'http\S+', '',row))
df_new['text']=df_new['text'].apply(lambda row : row.lower())
df_new['text']=df_new['text'].apply(lambda row : re.sub('\d+','',row))
df_new['text']=df_new['text'].apply(lambda row:re.sub(r"[^A-Za-z0-9']+", ' ',row))
df_new['text']=df_new['text'].apply(lambda row:re.sub(r'\s+', ' ',row))

Creating a Csv to get human annotations using Qualtrics survey

In [33]:
df_new.to_csv('Polls.csv', index = False, header=True)

Next we import the annotated new data and convert them into fastText readable form so that we can test our model over them.

In [34]:
df1 = pd.read_csv('Polls1.csv')
df2 = pd.read_csv('Polls2.csv')
df3 = pd.read_csv('Polls3.csv')
df4 = pd.read_csv('Polls4.csv')
df5 = pd.read_csv('Polls5.csv')

In [35]:
df1['label'] = '__label__' + df1['sentiment'].astype(str)
df1['lableled']=df1['label'].astype(str) +' '+ df1['text']
df2['label'] = '__label__' + df2['sentiment'].astype(str)
df2['lableled']=df2['label'].astype(str) +' '+ df2['text']
df3['label'] = '__label__' + df3['sentiment'].astype(str)
df3['lableled']=df3['label'].astype(str) +' '+ df3['text']
df4['label'] = '__label__' + df4['sentiment'].astype(str)
df4['lableled']=df4['label'].astype(str) +' '+ df4['text']
df5['label'] = '__label__' + df5['sentiment'].astype(str)
df5['lableled']=df5['label'].astype(str) +' '+ df5['text']

In [36]:
del df1['sentiment']
del df1['text']
del df1['label']
del df2['sentiment']
del df2['text']
del df2['label']
del df3['sentiment']
del df3['text']
del df3['label']
del df4['sentiment']
del df4['text']
del df4['label']
del df5['sentiment']
del df5['text']
del df5['label']

In [37]:
df1.to_csv('Poll1.csv', index = False, header=False)
df2.to_csv('Poll2.csv', index = False, header=False)
df3.to_csv('Poll3.csv', index = False, header=False)
df4.to_csv('Poll4.csv', index = False, header=False)
df5.to_csv('Poll5.csv', index = False, header=False)

In [38]:
csv_file = input('Input csv file Poll1.csv ')
txt_file = input('Input text file Poll1.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Input csv file name Poll1.csv
Input text file namePoll1.txt


In [39]:
csv_file = input('Input csv file Poll2.csv ')
txt_file = input('Input text file Poll2.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Input csv file Poll2.csv Poll2.csv
Input text file Poll2.txtPoll2.txt


In [40]:
csv_file = input('Input csv file Poll3.csv ')
txt_file = input('Input text file Poll3.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Input csv file Poll3.csv Poll3.csv
Input text file Poll3.txtPoll3.txt


In [41]:
csv_file = input('Input csv file Poll4.csv ')
txt_file = input('Input text file Poll4.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Input csv file Poll4.csv Poll4.csv
Input text file Poll4.txtPoll4.txt


In [42]:
csv_file = input('Input csv file Poll5.csv ')
txt_file = input('Input text file Poll5.txt')
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

Input csv file Poll5.csv Poll5.csv
Input text file Poll5.txtPoll5.txt
